### Import Necessary Packages

In [1]:
import pandas
import numpy as np
import os
import shutil
import torch

### Import Data

In [2]:
# labels = ["coca_cola","fanta","7up"]
# for label in labels:
#     os.makedirs(f'classification/train/{label}')
#     os.makedirs(f'classification/val/{label}')
#     os.makedirs(f'classification/test/{label}')

In [3]:
# cola_train_count = 0
# cola_val_count = 0
# cola_test_count = 0
# fanta_train_count = 0
# fanta_val_count = 0
# fanta_test_count = 0
# 7up_train_count = 0
# 7up_val_count = 0
# 7up_test_count = 0

# sub_dir = ["coca cola can", "coca cola plastic bottle", "fanta can", 
#            "fanta plastic bottle", "7up can", "7up plastic bottle"]
# for d in sub_dir:
#     if d.split()[0] == "coca":
#         for f in os.listdir(f'DATA/data/{d}'):
#             if cola_train_count < 800:
#                 shutil.copy(f'DATA/data/{d}/'+f,f'classification/train/coca_cola/{cola_train_count}.jpg')
#                 cola_train_count += 1
#             elif cola_val_count < 100:
#                 shutil.copy(f'DATA/data/{d}/'+f,f'classification/val/coca_cola/{cola_val_count}.jpg')
#                 cola_val_count += 1
#             elif cola_test_count < 100:
#                 shutil.copy(f'DATA/data/{d}/'+f,f'classification/test/coca_cola/{cola_test_count}.jpg')
#                 cola_test_count += 1
#     elif d.split()[0] == "fanta":
#         for f in os.listdir(f'DATA/data/{d}'):
#             if fanta_train_count < 800:
#                 shutil.copy(f'DATA/data/{d}/'+f,f'classification/train/fanta/{fanta_train_count}.jpg')
#                 fanta_train_count += 1
#             elif fanta_val_count < 100:
#                 shutil.copy(f'DATA/data/{d}/'+f,f'classification/val/fanta/{fanta_val_count}.jpg')
#                 fanta_val_count += 1
#             elif fanta_test_count < 100:
#                 shutil.copy(f'DATA/data/{d}/'+f,f'classification/test/fanta/{fanta_test_count}.jpg')
#                 fanta_test_count += 1
#     elif d.split()[0] == "7up":
#         for f in os.listdir(f'DATA/data/{d}'):
#             if 7up_train_count < 800:
#                 shutil.copy(f'DATA/data/{d}/'+f,f'classification/train/7up/{7up_train_count}.jpg')
#                 7up_train_count += 1
#             elif 7up_val_count < 100:
#                 shutil.copy(f'DATA/data/{d}/'+f,f'classification/val/7up/{7up_val_count}.jpg')
#                 7up_val_count += 1
#             elif 7up_test_count < 100:
#                 shutil.copy(f'DATA/data/{d}/'+f,f'classification/test/7up/{7up_test_count}.jpg')
#                 7up_test_count += 1

In [4]:
train_dir = 'classification/train'
val_dir = 'classification/val'
test_dir = 'classification/test'

### Data Process

In [5]:
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms, datasets
from collections import OrderedDict

In [6]:
dataset_transform = transforms.Compose([
  transforms.Resize(256),
  transforms.CenterCrop(224),
  transforms.RandomRotation(30),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
# Create train, validation and test dataset
train_dataset = datasets.ImageFolder(train_dir,transform=dataset_transform)
val_dataset = datasets.ImageFolder(val_dir,transform=dataset_transform)
test_dataset = datasets.ImageFolder(test_dir,transform=dataset_transform)

In [8]:
# Load datasets into dataloader for iteration
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 100, shuffle = True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 100, shuffle = True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle = True)

### Training process

In [9]:
# Training function
def training(model):
    iteration = 0
    minLoss = float("inf")
    best_parameters = None

    # Training Loop
    for epoch in range(epochs):
        # Iterate over the training dataloader
        for i, (images, labels) in enumerate(train_dataloader):
            # Reinitialize the gradient
            optimizer.zero_grad()

            # Get the correct label
            images, labels = images.to(device), labels.to(device)

            # Forward propagation
            outputs = model(images)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Use the loss for back propagation
            loss.backward()

            # Update network parameters
            optimizer.step()
            
            # Empty the cache of GPU
            del images
            del labels
            del loss
            del outputs
            torch.cuda.empty_cache()

            iteration += 1

            if iteration % 10 == 0 or iteration == 1:
                # Calculate Accuracy and loss        
                correct = 0
                total = 0

                total_loss = 0

                # Validation Loop
                # Iterate over the validation dataloader
                for images, labels in val_dataloader:

                    images, labels = images.to(device), labels.to(device)
                    # Make prediction
                    outputs = model(images)

                    loss = criterion(outputs, labels)
                    total_loss += float(loss)

                    # Get predictions from the maximum value, index 0 is the maximum value, 
                    # index 1 is the index of the maximum value
                    _, predicted = torch.max(outputs.data, 1)

                    # Number of labels
                    total += labels.size(0)

                    # Number of correct predictions
                    correct += (predicted == labels).sum()
                    
                    # Empty cache of GPU
                    del images
                    del labels
                    del loss
                    del outputs
                    torch.cuda.empty_cache()

                # Calculate accuracy
                accuracy = 100 * correct / total

                # Save parameters if loss is smaller
                if total_loss < minLoss:
                    minLoss = total_loss
                    best_parameters = model.state_dict()

                # Print accuracy and loss
                print(f'Iteration {iteration}: Loss: {total_loss}, Validation accuracy: {accuracy}')
    return best_parameters

In [10]:
import matplotlib.pyplot as plt
# Testing
def test(model):
    correct = 0
    total = 0
    total_loss = 0
    count = 0

    # Test Loop
    for images, labels in test_dataloader:

        images, labels = images.to(device), labels.to(device)

        # Make prediction
        outputs = model(images)

        loss = criterion(outputs, labels)
        total_loss += float(loss)

        # Get predictions from the maximum value, index 0 is the maximum value, 
        # index 1 is the index of the maximum value
        _, predicted = torch.max(outputs.data, 1)

        # Number of labels
        total += labels.size(0)

        # Number of correct predictions
        correct += (predicted == labels).sum()
        incorrect = images[predicted != labels]
        
        # Show 5 wrong predicted example
        if count < 5 and (predicted == labels).sum() != labels.size(0):
            
            wrong = "dog" if predicted[predicted != labels][0] else "cat"
            actual = "dog" if labels[predicted != labels][0] else "cat"
            print(f"Predicted {wrong}")
            print(f"Actual {actual}")
            show_image = incorrect[0].cpu().numpy().transpose((1, 2, 0))
            plt.imshow(show_image, cmap='gray')
            plt.show()
            count += 1
        
        # Empty cache
        del images
        del labels
        del loss
        del outputs
        torch.cuda.empty_cache()

    # Calculate accuracy
    accuracy = 100 * correct / total

    # Print accuracy and loss
    print(f'Best model has Loss: {total_loss}, Test accuracy: {accuracy}')

### Show prediction mistakes

In [11]:
labels = ['coca', 'fanta', '7-up']

In [12]:
def show_mistake(model, test_loader, show_count=5):
    count = 0
    for test_x, test_y in test_loader: 
        test_x, test_y = test_x.to(device), test_y.to(device)
        predict = model(test_x)
        # get the index of max predict as label
        _, predict_label = torch.max(predict.data, 1)
        for i in range(len(test_y)):
            if predict_label[i] != test_y[i]:
                count += 1
                if count > show_count:
                    return
                pred = labels[predict_label[i]]
                true_label = labels[test_y[i]]
                print('True Label: ', true_label)
                print('Prediction: ', pred)
#                 img = Image.open(test_x[0])
                img = torch.tensor(test_x[i], device = 'cpu')
#                 img = test_x[i]
                plt.imshow(img.permute(1, 2, 0))
                plt.axis('off')
                plt.show()

### VGG Model

In [13]:
VGG = models.vgg16(pretrained=True)
VGG

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [14]:
# Modify the last layer
VGG.classifier[6] = nn.Linear(in_features=4096, out_features=3)
# VGG.classifier.add_module("7", nn.Sigmoid())
# print(VGG.classifier)

In [15]:
# Only unfreeze the last two layer
to_update = ["classifier.6.weight", "classifier.6.bias"]
to_update_params = []
for name, param in VGG.named_parameters():
    if name in to_update:
        param.requires_grad = True
        to_update_params.append(param)
    else:
        param.requires_grad = False

In [16]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [17]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
VGG = VGG.to(device)

In [18]:
# Training
vgg_best_parameters = training(VGG)

C:\Users\Leo\anaconda3\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Iteration 1: Loss: 6.600369930267334, Validation accuracy: 37.66666793823242
Iteration 10: Loss: 3.9557217359542847, Validation accuracy: 67.66667175292969
Iteration 20: Loss: 3.8326992988586426, Validation accuracy: 71.0
Iteration 30: Loss: 2.994852900505066, Validation accuracy: 72.33333587646484
Iteration 40: Loss: 3.03998202085495, Validation accuracy: 71.66667175292969
Iteration 50: Loss: 2.910650670528412, Validation accuracy: 73.33333587646484
Iteration 60: Loss: 3.008993923664093, Validation accuracy: 73.66667175292969
Iteration 70: Loss: 2.5159011483192444, Validation accuracy: 75.0
Iteration 80: Loss: 3.3168163299560547, Validation accuracy: 72.33333587646484
Iteration 90: Loss: 2.9726046323776245, Validation accuracy: 72.0
Iteration 100: Loss: 2.8574482202529907, Validation accuracy: 75.66667175292969
Iteration 110: Loss: 3.550526261329651, Validation accuracy: 72.33333587646484
Iteration 120: Loss: 3.3570820093154907, Validation accuracy: 71.66667175292969
Iteration 130: Lo

In [19]:
import os
ModelPath = 'best_models'
if not os.path.exists(ModelPath):
    os.makedirs(ModelPath)

In [20]:
torch.save(vgg_best_parameters, ModelPath+'/vgg_params.pt')

### AlexNet Model

In [21]:
alexnet = models.alexnet(pretrained=True)
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [22]:
alexnet.classifier[6] = nn.Linear(in_features=4096, out_features=3)

In [23]:
# Only unfreeze the last two layer
to_update = ["classifier.6.weight", "classifier.6.bias"]
to_update_params = []
for name, param in alexnet.named_parameters():
    if name in to_update:
        param.requires_grad = True
        to_update_params.append(param)
    else:
        param.requires_grad = False

In [24]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [25]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

In [26]:
# Training
alexnet_best_parameters = training(alexnet)

Iteration 1: Loss: 20.435325622558594, Validation accuracy: 35.66666793823242
Iteration 10: Loss: 5.7077600955963135, Validation accuracy: 74.0
Iteration 20: Loss: 5.720728635787964, Validation accuracy: 71.0
Iteration 30: Loss: 3.014269769191742, Validation accuracy: 82.66667175292969
Iteration 40: Loss: 3.3988410234451294, Validation accuracy: 83.66667175292969
Iteration 50: Loss: 3.297947883605957, Validation accuracy: 81.66667175292969
Iteration 60: Loss: 2.502429783344269, Validation accuracy: 88.0
Iteration 70: Loss: 1.2752601504325867, Validation accuracy: 91.0
Iteration 80: Loss: 2.294757068157196, Validation accuracy: 85.66667175292969
Iteration 90: Loss: 2.7286709547042847, Validation accuracy: 83.66667175292969
Iteration 100: Loss: 2.0716536939144135, Validation accuracy: 88.0
Iteration 110: Loss: 2.163800895214081, Validation accuracy: 87.0
Iteration 120: Loss: 2.6734888553619385, Validation accuracy: 82.66667175292969
Iteration 130: Loss: 2.11293488740921, Validation accur

In [27]:
# save the model params
torch.save(alexnet_best_parameters, ModelPath+'/alexnet_params.pt')

### GoogleNet

In [28]:
googlenet = models.googlenet(pretrained=True)
googlenet

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [29]:
googlenet.fc = nn.Linear(in_features=1024, out_features=3, bias=True)

In [30]:
# Only unfreeze the last two layer
to_update = ["fc.weight", "fc.bias"]
to_update_params = []
for name, param in googlenet.named_parameters():
    if name in to_update:
        param.requires_grad = True
        to_update_params.append(param)
    else:
        param.requires_grad = False

In [31]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [32]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
googlenet = googlenet.to(device)

In [33]:
# Training
googlenet_best_parameters = training(googlenet)

Iteration 1: Loss: 8.913947820663452, Validation accuracy: 33.333335876464844
Iteration 10: Loss: 3.7672622203826904, Validation accuracy: 35.0
Iteration 20: Loss: 2.4132933020591736, Validation accuracy: 64.33333587646484
Iteration 30: Loss: 2.100605010986328, Validation accuracy: 69.66667175292969
Iteration 40: Loss: 2.143932282924652, Validation accuracy: 69.66667175292969
Iteration 50: Loss: 1.7346675992012024, Validation accuracy: 75.33333587646484
Iteration 60: Loss: 1.6152511239051819, Validation accuracy: 78.33333587646484
Iteration 70: Loss: 1.7114620208740234, Validation accuracy: 75.0
Iteration 80: Loss: 1.607013612985611, Validation accuracy: 79.66667175292969
Iteration 90: Loss: 1.7011368870735168, Validation accuracy: 78.33333587646484
Iteration 100: Loss: 1.6173535585403442, Validation accuracy: 80.66667175292969
Iteration 110: Loss: 1.5043530464172363, Validation accuracy: 79.33333587646484
Iteration 120: Loss: 2.0698322653770447, Validation accuracy: 73.33333587646484


In [34]:
# save the model params
torch.save(googlenet_best_parameters, ModelPath+'/googlenet_params.pt')

### ResNet

In [35]:
resnet18 = models.resnet18(pretrained=True)
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [36]:
resnet18.fc = nn.Linear(in_features=512, out_features=3, bias=True)

In [37]:
# Only unfreeze the last two layer
to_update = ["fc.weight", "fc.bias"]
to_update_params = []
for name, param in resnet18.named_parameters():
    if name in to_update:
        param.requires_grad = True
        to_update_params.append(param)
    else:
        param.requires_grad = False

In [38]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [39]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = resnet18.to(device)

In [40]:
# Training
resnet18_best_parameters = training(resnet18)

Iteration 1: Loss: 13.193704605102539, Validation accuracy: 33.333335876464844
Iteration 10: Loss: 6.038305163383484, Validation accuracy: 33.66666793823242
Iteration 20: Loss: 3.102333664894104, Validation accuracy: 55.333335876464844
Iteration 30: Loss: 2.07155042886734, Validation accuracy: 74.0
Iteration 40: Loss: 2.1329798698425293, Validation accuracy: 76.0
Iteration 50: Loss: 2.066888988018036, Validation accuracy: 78.33333587646484
Iteration 60: Loss: 1.7342987060546875, Validation accuracy: 81.66667175292969
Iteration 70: Loss: 1.6970323622226715, Validation accuracy: 79.0
Iteration 80: Loss: 1.7206554412841797, Validation accuracy: 81.0
Iteration 90: Loss: 1.6863808929920197, Validation accuracy: 77.66667175292969
Iteration 100: Loss: 1.6108253002166748, Validation accuracy: 81.66667175292969
Iteration 110: Loss: 1.3255358040332794, Validation accuracy: 82.0
Iteration 120: Loss: 1.5978454947471619, Validation accuracy: 81.0
Iteration 130: Loss: 1.5504722893238068, Validation 

In [41]:
# save the model params
torch.save(resnet18_best_parameters, ModelPath+'/resnet18_params.pt')

### MobileNet V3 Small

In [42]:
mobilenet_v3_small = models.mobilenet_v3_small(pretrained=True)
mobilenet_v3_small

MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride

In [43]:
mobilenet_v3_small.classifier[3] = nn.Linear(in_features=1024, out_features=3, bias=True)

In [44]:
# Only unfreeze the last two layer
to_update = ["classifier.3.weight", "classifier.3.bias"]
to_update_params = []
for name, param in mobilenet_v3_small.named_parameters():
    if name in to_update:
        param.requires_grad = True
        to_update_params.append(param)
    else:
        param.requires_grad = False

In [45]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [46]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mobilenet_v3_small = mobilenet_v3_small.to(device)

In [47]:
# Training
mobilenet_v3_small_best_parameters = training(mobilenet_v3_small)

Iteration 1: Loss: 2.5571563839912415, Validation accuracy: 68.0
Iteration 10: Loss: 1.6668530702590942, Validation accuracy: 79.66667175292969
Iteration 20: Loss: 1.516827404499054, Validation accuracy: 81.33333587646484
Iteration 30: Loss: 1.1859776079654694, Validation accuracy: 86.0
Iteration 40: Loss: 1.1248865723609924, Validation accuracy: 87.66667175292969
Iteration 50: Loss: 1.0330188572406769, Validation accuracy: 89.0
Iteration 60: Loss: 1.014381468296051, Validation accuracy: 87.66667175292969
Iteration 70: Loss: 0.9033661484718323, Validation accuracy: 90.0
Iteration 80: Loss: 1.0003553926944733, Validation accuracy: 88.0
Iteration 90: Loss: 0.9582179188728333, Validation accuracy: 91.0
Iteration 100: Loss: 0.8809913694858551, Validation accuracy: 90.0
Iteration 110: Loss: 1.0092419683933258, Validation accuracy: 88.33333587646484
Iteration 120: Loss: 0.7209042608737946, Validation accuracy: 92.0
Iteration 130: Loss: 0.8710432648658752, Validation accuracy: 89.333335876464

In [48]:
# save the model params
torch.save(mobilenet_v3_small_best_parameters, ModelPath+'/mobilenet_v3_small_params.pt')

### RegNet 16

In [55]:
regnet_x_16gf = models.regnet_x_16gf(pretrained=True)
regnet_x_16gf

RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): ConvNormActivation(
          (0): Conv2d(32, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): ConvNormActivation(
            (0): Conv2d(32, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): ConvNormActivation(
            (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=2, bias=False)
            

In [56]:
regnet_x_16gf.fc = nn.Linear(in_features=2048, out_features=3, bias=True)

In [57]:
# Only unfreeze the last two layer
to_update = ["fc.weight", "fc.bias"]
to_update_params = []
for name, param in regnet_x_16gf.named_parameters():
    if name in to_update:
        param.requires_grad = True
        to_update_params.append(param)
    else:
        param.requires_grad = False

In [58]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [59]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
regnet_x_16gf = regnet_x_16gf.to(device)

In [60]:
# Training
regnet_x_16gf_best_parameters = training(regnet_x_16gf)

Iteration 1: Loss: 21.541993141174316, Validation accuracy: 33.333335876464844
Iteration 10: Loss: 3.1268529891967773, Validation accuracy: 54.333335876464844
Iteration 20: Loss: 2.973981499671936, Validation accuracy: 63.0
Iteration 30: Loss: 2.377226769924164, Validation accuracy: 68.33333587646484
Iteration 40: Loss: 1.7876751124858856, Validation accuracy: 77.33333587646484
Iteration 50: Loss: 1.673594892024994, Validation accuracy: 81.66667175292969
Iteration 60: Loss: 1.406602531671524, Validation accuracy: 82.33333587646484
Iteration 70: Loss: 1.234171599149704, Validation accuracy: 85.0
Iteration 80: Loss: 1.742649257183075, Validation accuracy: 79.33333587646484
Iteration 90: Loss: 1.3638516068458557, Validation accuracy: 80.66667175292969
Iteration 100: Loss: 1.138173907995224, Validation accuracy: 88.33333587646484
Iteration 110: Loss: 1.3834729492664337, Validation accuracy: 81.0
Iteration 120: Loss: 1.0195235311985016, Validation accuracy: 86.33333587646484
Iteration 130: 

In [61]:
# save the model params
torch.save(regnet_x_16gf_best_parameters, ModelPath+'/regnet_x_16gf_params.pt')

### Expriment on customized CNN model

In [62]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        self.cnn_layers = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        # Fully connected output
        self.linear_layers = nn.Sequential(
            nn.Linear(100352, 3)
        )
 
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [63]:
CNNmodel = CNNModel()

In [64]:
CNNmodel

CNNModel(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=100352, out_features=3, bias=True)
  )
)

In [65]:
to_update_params = []
for name, param in CNNmodel.named_parameters():
    to_update_params.append(param)

In [66]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [67]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CNNmodel = CNNmodel.to(device)

In [68]:
# Training
CNN_best_parameters = training(CNNmodel)

Iteration 1: Loss: 543.3722229003906, Validation accuracy: 40.0
Iteration 10: Loss: 115.72663688659668, Validation accuracy: 57.66666793823242
Iteration 20: Loss: 50.50222396850586, Validation accuracy: 65.33333587646484
Iteration 30: Loss: 56.401442527770996, Validation accuracy: 60.0
Iteration 40: Loss: 36.218241691589355, Validation accuracy: 66.33333587646484
Iteration 50: Loss: 47.52525329589844, Validation accuracy: 63.66666793823242
Iteration 60: Loss: 42.01823902130127, Validation accuracy: 69.0
Iteration 70: Loss: 35.701608657836914, Validation accuracy: 70.33333587646484
Iteration 80: Loss: 28.68470287322998, Validation accuracy: 70.66667175292969
Iteration 90: Loss: 37.42268466949463, Validation accuracy: 66.33333587646484
Iteration 100: Loss: 42.41492176055908, Validation accuracy: 63.0
Iteration 110: Loss: 30.076029777526855, Validation accuracy: 68.33333587646484
Iteration 120: Loss: 37.61164569854736, Validation accuracy: 65.66667175292969
Iteration 130: Loss: 23.9269895

In [69]:
# save the model params
torch.save(CNN_best_parameters, ModelPath+'/CNN_params.pt')

In [70]:
class CNNModel2(nn.Module):
    def __init__(self):
        super(CNNModel2, self).__init__()
        self.cnn_layers = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )
        # Fully connected output
        self.linear_layers = nn.Sequential(
            nn.Linear(100352, 3)
        )

    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [71]:
CNNmodel2 = CNNModel2()

In [72]:
CNNmodel2

CNNModel2(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=100352, out_features=3, bias=True)
  )
)

In [73]:
to_update_params = []
for name, param in CNNmodel2.named_parameters():
    to_update_params.append(param)

In [74]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [75]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CNNmodel2 = CNNmodel2.to(device)

In [76]:
# Training
CNN2_best_parameters = training(CNNmodel2)

Iteration 1: Loss: 1160.4410400390625, Validation accuracy: 33.333335876464844
Iteration 10: Loss: 128.91955184936523, Validation accuracy: 54.66666793823242
Iteration 20: Loss: 173.16294479370117, Validation accuracy: 48.333335876464844
Iteration 30: Loss: 83.51352882385254, Validation accuracy: 54.333335876464844
Iteration 40: Loss: 44.0849494934082, Validation accuracy: 59.66666793823242
Iteration 50: Loss: 33.612064361572266, Validation accuracy: 58.0
Iteration 60: Loss: 21.88612651824951, Validation accuracy: 62.0
Iteration 70: Loss: 24.13911485671997, Validation accuracy: 62.66666793823242
Iteration 80: Loss: 27.9541654586792, Validation accuracy: 63.0
Iteration 90: Loss: 34.74744415283203, Validation accuracy: 63.333335876464844
Iteration 100: Loss: 19.62210512161255, Validation accuracy: 68.33333587646484
Iteration 110: Loss: 20.615886211395264, Validation accuracy: 62.333335876464844
Iteration 120: Loss: 11.148518085479736, Validation accuracy: 69.0
Iteration 130: Loss: 13.476

In [77]:
# save the model params
torch.save(CNN2_best_parameters, ModelPath+'/CNN2_params.pt')